```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{pyproject.toml, packaging, and you}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

I live in Belmont,
in the San Francisco Bay Area Peninsula.
I wish to acknowledge it as the
ancestral homeland
of the
Ramaytush Ohlone people.

```{=latex}
\begin{frame}[fragile]
\frametitle{Matching Donation: Encircle}

LGBTQ+ Charity in Utah

\verb|https://encircletogether.org/|

Matching up to total of \$100

\end{frame}
```

Encircle is a charity
supporting LGBTQ+ youth.
I will be matching donations up to
`$100`.
E-mail or message me receipts.

## Basics

### What is TOML?

```{=latex}
\begin{frame}
\frametitle{What is TOML?}

Semantics: \pause JSON + date + float vs. integer \pause

Syntax: \pause more editable than JSON, \pause easier to parse than YAML

\end{frame}
```

TOML is a format for structured data.
As far as
*semantics*
go,
TOML can express what JSON can and a little bit more.

TOML has a native
"date"
type.
It can also express which numbers are supposed to be
integers and which are supposed to be floats.

TOML is designed to be more editable than JSON.
It allows for comments
and trailing commas,
for example.

It is also designed to be easier to parse than YAML.
The specification is shorter,
and tries to be fairly clear.

```{=latex}
\begin{frame}[fragile]
\frametitle{TOML example}
```

In [1]:
toml_stuff = """\
[project] # Table -> Dictionary
name = "orbipatch"
authors = [ # Array -> List
# Key/Value -> Dictionary
{ name = "MZ", email = "mz@devskillup.com" }
]
"""

```{=latex}
\end{frame}
```

A simple example of TOML,
sampled from
`pyproject.toml`,
can help illustrate some of these.
Note that we can have
`#`-style
comments.

Tables,
as well as
Key/Value
syntax,
parse into dictionaries.
There are also arrays,
which can include arbitrary objects.

```{=latex}
\begin{frame}[fragile]
\frametitle{TOML example}
```

In [35]:
import tomli # tomllib in Python 3.11+
import json
print(json.dumps(
    tomli.loads(toml_stuff),
    indent=4,
))

{
    "project": {
        "name": "orbipatch",
        "authors": [
            {
                "name": "MZ",
                "email": "mz@devskillup.com"
            }
        ]
    }
}


```{=latex}
\end{frame}
```

The
`tomli`
library
is similar to the built-in
`tomllib`
in Python 3.11 and above.
Parsing the data above,
and redumping it into JSON,
helps show the structure.

### What is pyproject.toml?

```{=latex}
\begin{frame}
\frametitle{pyproject.toml}

Originally: Configure build system, \pause
experimenting with alternatives, \pause
setuptools plugins, \pause
etc. \pause

Now: \pause
Still that but also: \pause
build-system agnostic metadata, \pause
configure ecosystem tools.

\end{frame}
```

The
`pyproject.toml`
file was originally intended to configure the build system.
This had to be outside of
`setup.py`,
in order to install the right version of
`setuptools`
and
any plugins it depended on.

This is also important to allow other build systems.
Allowing alternative build systems,
without the backwards-compatibility needs of setuptools,
allows faster experimentation.

However,
because of
TOML's
flexibility,
it was quickly adopted into other tools.
Many tools used the fact that
`tool.<toolname>.<section>`
would all be collected into
`toml_data["tool"]["toolname"]`.

Finally,
some of the fields were standardized.
This led to the
`project`
section,
which is a standard way to express standard metadata about Python packages.

### Parsing pyproject.toml

In [36]:
minimal_pyproject = """\
[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]
"""

This is not technically a
"minimal"
pyproject,
but it is fairly minimalistic.
The
`build-system`
configures the how to build the package.

The
`requires`
configures
the build-time
*dependencies*.
The
*backend*
is the path to a module
which has some functions,
most importantly
`build_wheel`.

The parameters in
`project`
correspond to some often that
used to be set in
`setup.py`.
More importantly,
these are fields which will end up
being in a wheel.

```{=latex}
\begin{frame}[fragile]
\frametitle{Parsing pyproject.toml}
```

In [37]:
parsed = tomli.loads(minimal_pyproject)
print(
    "Build system requires:",
    parsed["build-system"].pop("requires")
)

Build system requires: ['setuptools']


```{=latex}
\end{frame}
```

When parsing the TOML,
sections as well as fields within sections
become dictionaries.
The result is the typical JSON-like dict-within-dict nested structure.

The
`build-system`
section has two important keys.
The first one is the
`requires`.
More serious
`requires`
will sometimes pin the version.

```{=latex}
\begin{frame}[fragile]
\frametitle{Parsing pyproject.toml}
```

In [12]:
print(
    "Build backend",
    parsed["build-system"].pop("build-backend")
)

Build backend setuptools.build_meta


```{=latex}
\end{frame}
```

The other field is the
`build-system`.
This is a module,
typically installed by
one of the modules in
`requires`,
which knows how to build wheels.

The default backend,
here used explicitly,
it the
`setuptools`
one.
The
`setuptools`
package exposes its
build-system-compatible module
as
`build_meta`.

```{=latex}
\begin{frame}[fragile]
\frametitle{Parsing pyproject.toml}
```

In [13]:
print(
    "Project authors:\n",
    parsed["project"].pop("authors")
)

Project authors:
 [{'name': 'MZ', 'email': 'mz@devskillup.com'}]


```{=latex}
\end{frame}
```

The
`project`
section of the
`pyproject.toml`
file has several interesting metadata fields.
Of special interest is the
`authors`
one,
particularly because of its complexity:
it is an
*array*
of
*key-value* pairs.

Accessing the name of the first author
is a good example of how deep the nesting
here is.
It would be
`parsed["project"]["authors"][0]["name"]`,
for a total of four nesting levels.

```{=latex}
\begin{frame}[fragile]
\frametitle{Parsing pyproject.toml}
```

In [14]:
print(
    "Project description:\n",
    parsed["project"].pop("description"),
)

Project description:
 silly project named after a niche math thing


```{=latex}
\end{frame}
```

The description is interesting in that even when it is a short description,
it is often objectively long.
Using a regular string is fine for it,
but TOML does support triple quoted strings if the need arises.

```{=latex}
\begin{frame}[fragile]
\frametitle{Parsing pyproject.toml}
```

In [17]:
print(
    "Project:\n",
    json.dumps(parsed.pop("project"), indent=2),
)

Project:
 {
  "name": "orbipatch",
  "version": "2022.3.6.2",
  "readme": "README.rst"
}


```{=latex}
\end{frame}
```

The other fields in the project section are some of the most important ones. In particular, name and version make it not only to the contents of the wheel, but also to its *name*.

In theory,
`pyproject.toml`
allows inlining the README,
or long description.
There are good reasons to avoid this
"saving".

First,
this makes it hard to read natively.
Even worse,
this won't render on your favorite
code-collaboration platform at *all*.

Luckily,
you can include it by reference.
This allows unifying the project's README
on
GitHub or similar
with the description.


### Build system

```{=latex}
\begin{frame}
\frametitle{build system}

requires: \pause
Dependencies \pause

backend: \pause
Module that has the right methods

\end{frame}
```

The
`build-system`
has the build
*dependencies*
and the
*backend*.
Together,
they determine how to build the package
into a wheel.

### Project

```{=latex}
\begin{frame}
\frametitle{project}

Must: \pause
name, version \pause

Recommended: \pause
Short description (usually inline), \pause
Long description (usually from file), \pause
License (usually from file, can be inlined), \pause
URLs (especially "Homepage")

\end{frame}
```

The project section is interesting.
It is not
*required*
to be there.

If it
*does*
exist,
it must have the name and version fields.
Other fields are highly recommended.

Some of the other fields will be displayed
directly in PyPI.
For example,
this is the case with the URLs
and the description.

One interesting file to inline is the
license.
This reduces by one the number of files
at your top level.

## Extensibility

### The tools section

In [35]:
minimal_pyproject = """\
[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[tool.black]
include = '\.pyi'
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{The tools section}

Under "tool.NAME"
```

In [6]:
# [tool.black]
# include = '\.pyi'
print(
    "Black configuration:",
    parsed["tool"]["black"],
)

Black configuration: {'include': '\\.pyi'}


```{=latex}
\end{frame}
```

Many,
though not all,
Python ecosystem tools support
pyproject-based configuration.
For example,
`black`
will allow you to configure which
files it should check.

### Example: Configuring coverage

In [56]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[tool.coverage.run]
branch = true
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring coverage}

Like with setup.cfg, with prefix "tool.":
```

In [58]:
# [tool.coverage.run]
# branch = true
print(
    "Coverage configuration:",
    parsed["tool"]["coverage"],
)

Coverage configuration: {'run': {'branch': True}}


```{=latex}
\end{frame}
```

Coverage can also be configured via pyproject.
In this example,
we are configuring branch coverage
(as you should!)

### Example: Configuring isort

In [20]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[tool.isort]
src_paths = ["isort", "test"]
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring isort}

Like with setup.cfg, with prefix "tool.":
```

In [21]:
# [tool.isort]
# src_paths = ["isort", "test"]
print(
    "isort configuration:\n",
    parsed["tool"]["isort"],
)

isort configuration:
 {'src_paths': ['isort', 'test']}


```{=latex}
\end{frame}
```

The last Python ecosystem tool I'll show is
`isort`.
In this example,
it shows how to configure the source paths
that isort will cover.

## Project metadata

```{=latex}
\begin{frame}
\frametitle{project metadata}

project section: \pause
packaging semantics edition

\end{frame}
```

Some project metadata is
"semantically neutral".
Python doesn't care if your project is called
"super_cool"
or
"really_sad".

Other fields in the project metadata are less so.
Let's dive into them!

### Dependencies

In [64]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]
dependencies = ["six"]
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring dependencies}

```

In [ ]:
# [project]
# ...
# dependencies = ["six"]
print(
    "dependencies:",
    parsed["project"]["dependencies"],
)

```{=latex}
\end{frame}
```

The most common
semantically meaningful
field is dependencies.
It is not universal:
some packages do not depend on any others as a matter of policy.

For everything else,
the 99%,
the
`dependencies`
field in the metadata does the trick.
Those dependencies will be translated into
wheel
dependencies by most packaging backends,
including
`setuptools`.

### Extra dependencies

In [31]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[project.optional-dependencies]
tests = ["pytest"]
docs = ["sphinx"]
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring optional dependencies}

```

In [32]:
# [project.optional-dependencies]
# tests = ["pytest"]
# docs = ["sphinx"]
print(
    "Optional dependencies:\n",
    parsed["project"]["optional-dependencies"],
)

Optional dependencies:
 {'tests': ['pytest'], 'docs': ['sphinx']}


```{=latex}
\end{frame}
```

Second only to the regular dependencies
come the
"optional"
dependencies.
Whether it is
dev-only
dependencies
or dependencies that support a particular way to use the package,
these are everywhere.

In theory,
you could use a KV
format with
`optional-dependnecies = { tests = ["pytest"] }`
but this feels a little on the busy side.

### Console scripts

In [73]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[project.scripts]
awesome-command = "my_package:main"
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring console scripts}

```

In [74]:
# [project.scripts]
# awesome-command = "my_package:main"
print(
    "scripts:",
    parsed["project"]["scripts"],
)

scripts: {'awesome-command': 'my_package:main'}


```{=latex}
\end{frame}
```

Next in the list of popularity is
"console scripts".
These used to be part of
`entrypoints`
but got so big they deserve their own section.

The semantics are
`full.path.to.module:callable_object`.
The wheel builder will take care of the rest.

### Entrypoints

In [33]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
version = "2022.3.6.2"
description = "silly project named after a niche math thing"
readme = "README.rst"
authors = [{name = "MZ", email = "mz@devskillup.com"}]

[project.entry-points."paste.app_factory"]
main = "my-package:main"
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuring entry points}

```

In [34]:
# [project.entry-points."paste.app_factory"]
# main = "my-package:main"
print(
    "entry points:\n",
    parsed["project"]["entry-points"],
)

entry points:
 {'paste.app_factory': {'main': 'my-package:main'}}


```{=latex}
\end{frame}
```

Generic entrypoints are not gone,
though.
Your hand-crafted entrypoints for configuring
`paste`
or including a setuptools plugins
are still working.


In theory, the semantics are arbitrary,
but the convention of
`module:object`
is still strong.

## Building with setuptools

### Build system details

```{=latex}
\begin{frame}[fragile]
\frametitle{Build system}

Requires: \pause
Usual dependency rules (can include minimal, pinned, etc.) \pause

Build system: \pause
PEP 517:
```

In [75]:
def build_wheel(
    wheel_directory,
    config_settings=None,
    metadata_directory=None,
):
    ...

```{=latex}
\end{frame}
```

PEP517 describes how a build system is supposed to work.
There are a few optional parts,
but the most important part is a
`build_wheel`
function.
This function is similar to WSGI.

A
*builder*
is supposed to call this function with appropriate parameters.
A
*build system*
is supposed to implement the function correctly.

This allows multiple build front ends and multiple build back ends.
While there are many popular back ends,
just like in web servers,
front ends are fewer.

### Using `python -m build`

```{=latex}
\begin{frame}[fragile]
\frametitle{Packaging Python}

\includegraphics[scale=0.1]{package}

\end{frame}
```

The
`pyproject.toml`
file is useful in building
packages.

```{=latex}
\begin{frame}[fragile]
\frametitle{python -m build}

Usually: \pause Just works \pause

Hint: \pause Use "src/" structure \pause

Sometimes: \pause BETA! \pause
"tools.setup.\emph{something}"

\end{frame}
```

The most popular builder front end is
`-m build`.
It mostly just works with a correct
`pyproject.toml`
which configures
`setuptools`.

The most common failure with setuptools
is that it will fail to recognize where your sources are.
The best way to fix it is to use the
`src/`
layout,
which removes a lot of the ambiguity.

The alternative is to have a section for
`tools.setup.<something>`
to configure it explicitly.
Note that the fields and their semantics
are currently
"beta"
as far as setuptools is concerned.

### Editable installs with setuptools

```{=latex}
\begin{frame}[fragile]
\frametitle{Editable installs}

Empty setup.cfg: no longer needed

\includegraphics[scale=0.1]{editable}

\end{frame}
```

For
*editable*
installs,
you do need an empty
`setup.cfg`.
Luckily,
this is all you need.

Hopefully,
at some point,
this will no longer be needed.

### Dynamic fields

In [76]:
minimal_pyproject = """\
[build-system]
requires = ["setuptool"]
build-backend = "setuptools.build_meta"

[project]
name = "orbipatch"
dynamic = ["version"]
"""
parsed = tomli.loads(minimal_pyproject)

```{=latex}
\begin{frame}[fragile]
\frametitle{Dynamic fields}
```

In [79]:
# [project]
# name = "orbipatch"
# dynamic = ["version"]
print(
    "name",
    parsed["project"].pop("name"),
)
print(
    "project",
    parsed["project"],
)

name orbipatch
project {'dynamic': ['version']}


```{=latex}
\end{frame}
```

Earlier,
I said the project
*has*
to include name and version.
This was a mild lie.

A project has to include the version
*or*
explicitly note that it will be
filled in by other tools.
This is what the
`dynamic`
field does.
This allows using your wheel builder,
in our case setuptools,
to set the version.

### Example: Using `setuptools_scm`

```{=latex}
\begin{frame}[fragile]
\frametitle{setuptools scm}
```

In [80]:
# [build-system]
# requires = [
#   "setuptools",
#   "setuptools_scm",
# ]
#
# [project]
# name = "orbipatch"
# dynamic = ["version"]

```{=latex}
\end{frame}
```

Setuptools will not set the version itself,
though.
However,
there are plugins which allow setting the version.

One popular plugin is
`setuptools_scm`,
which will set the version from tags and other
aspects of the version-controlled repository.

## Recap

### pyproject.toml: source of truth

```{=latex}
\begin{frame}[fragile]
\frametitle{pyproject.toml}

Packaging! \pause

Also: Everything else \pause

Support in your own tooling
\end{frame}
```

`pyproject.toml`
can be the ultimate source of truth.
As more and more tools use it,
including packaging,
you need fewer
"boilerplate"
files.

If you write your own tooling for Python,
consider supporting
`pyproject.toml`
as your configuration file.

### Important fields

```{=latex}
\begin{frame}[fragile]
\frametitle{project fields}

Name
\pause

Version
\pause

Description, license, readme
\pause

Dependencies (and Optional dependencies)
\pause

Scripts
\pause

Entry points
\end{frame}
```

The important fields in
`pyproject.toml`'s `project` section are:

* Name
* Version
* Person facing metadata: Description, license, README
* Dependencies and optional dependencies
* Scripts and entrypoints

### setuptools support

```{=latex}
\begin{frame}[fragile]
\frametitle{setuptools support}

Defaults usually good \pause

Use src/ \pause
(convention over configuration!)
\pause

Configure lightly where you must \pause

"python -m build"
future-proofing

\end{frame}
```

With a reasonably-written
`pyproject.toml`,
setuptools will usually 
do the right thing
without any specific configuration.
This is especially if you use the less
ambiguous
`src/`
layout.

Where you need to configure,
tread lightly.
In your build tooling,
use
`python -m build`.
This makes switching to
flit,
hatch,
poetry,
or something else
possible by changing nothing more
than
`pyproject.toml`.

## Open Space


```{=latex}
\begin{frame}[fragile]
\frametitle{Open Space}

Awesome pyproject.toml \pause

Right now: 250A \pause

Book giveaway: DevOps in Python, Expert Twisted

\end{frame}
```

If you had fun,
why not join me at the open space
"awesome pyproject.toml"?
I'll be giving away signed books.

```{=latex}
\end{document}
```